# STA130 Course Project - Analysis 1 Project Proposal
***
#### **Author**: David Daniliuc<br>**Created**: Sat Nov. 2, 2024

*Jupyter Python Notebook for Analysis 1 testing and experimenting for the STA130 Course Project.* 

In [4]:

# Import statements
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
import statsmodels.formula.api as smf

# Read and import dataset
cols = pd.read_csv("var_names.csv")
data = pd.read_csv("CSCS_data_anon.csv", na_values=["9999", "", " ", "Presented but no response", "NA"])

# Cleaning the data
empty = (data.isna().sum() == data.shape[0])
data = data[empty.index[~empty]] # keep non empty columns only

data = data[data.REMOVE_case=='No'].copy()

data

/var/folders/b0/20_s9xf13p33yqxth9cv22380000gn/T/ipykernel_1634/1313995991.py:10: DtypeWarning: Columns (129,408,630,671,689,978,1001,1002,1006,1007,1008,1080,1113,1115,1116,1117,1118,1119,1120,1121,1124,1125,1126,1127,1128,1213,1214,1215,1216,1217,1218,1263,1266,1342,1343,1344,1345,1346,1347,1348,1349,1390,1391,1393,1439,1442,1463,1546,1549,1552,1555,1558,1561) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("CSCS_data_anon.csv", na_values=["9999", "", " ", "Presented but no response", "NA"])


,UNIQUE_id,UNIQUE_num_records,ELIGIBLE_consent,GEO_residence_canada,GEO_province,DEMO_age,DEMO_gender,DEMO_identity_vetrans,DEMO_identity_indigenous,DEMO_identity_lgbtq,...,PSYCH_body_self_image_questionnaire_height_dissatisfaction_score,PSYCH_body_self_image_questionnaire_fatness_evaluation_score,PSYCH_body_self_image_questionnaire_negative_affect_score,PSYCH_body_self_image_questionnaire_social_dependence_score,PSYCH_big_five_inventory_agreeable_score,PSYCH_big_five_inventory_conscientious_score,PSYCH_big_five_inventory_extraverted_score,PSYCH_big_five_inventory_neurotic_score,PSYCH_big_five_inventory_open_score,REMOVE_case
0,cscs_00001,1,Yes,Yes,British Columbia,71.0,Non-binary,NaN,NaN,"Sexual or gender minorities (e.g., LGBTQ2+)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,cscs_00002,1,Yes,Yes,Ontario,69.0,Woman,NaN,NaN,Not Selected,...,3.0,8.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,No
2,cscs_00003,1,Yes,Yes,Quebec,56.0,Woman,NaN,NaN,Not Selected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,cscs_00005,1,Yes,Yes,NaN,54.0,Woman,NaN,NaN,Not Selected,...,NaN,NaN,NaN,NaN,28.0,34.0,30.0,32.0,37.0,No
4,cscs_00006,1,Yes,Yes,Ontario,30.0,Man,Not Selected,"Indigenous peoples (e.g., First Nations, Métis...","Sexual or gender minorities (e.g., LGBTQ2+)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11425,cscs_11808,1,Yes,Yes,NaN,45.0,Woman,NaN,NaN,Not Selected,...,NaN,NaN,NaN,NaN,26.0,23.0,18.0,31.0,37.0,No
11426,cscs_11809,1,Yes,Yes,NaN,45.0,Woman,NaN,NaN,"Sexual or gender minorities (e.g., LGBTQ2+)",...,NaN,NaN,NaN,NaN,31.0,33.0,33.0,13.0,39.0,No
11427,cscs_11810,1,Yes,Yes,British Columbia,36.0,Man,NaN,NaN,Not Selected,...,NaN,NaN,NaN,NaN,32.0,37.0,31.0,NaN,38.0,No
11428,cscs_11812,3,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [5]:
### The Correlation Between Socioeconomic Status (SES) and Mental Health During the 2021 Lockdown of the COVID-19 Pandemic
important_columns = [
	'WELLNESS_life_satisfaction',
	'DEMO_household_income',
]

data_cross_2021 = data[data.DATASET=='2021 Cross-Sectional'][important_columns].copy().dropna()

data_cross_2021

,WELLNESS_life_satisfaction,DEMO_household_income
4,8.0,"$80,000 to $89,999"
8,1.0,"$20,000 to $24,999"
10,7.0,"$90,000 to $99,999"
30,1.0,"$5,000 to $9,999"
31,5.0,"$10,000 to $14,999"
...,...,...
11393,4.0,"Under $5,000"
11396,3.0,"$15,000 to $19,999"
11398,7.0,"$35,000 to $39,999"
11410,8.0,"$70,000 to $79,999"


In [6]:
income_mapping = {
    'Under $5,000': 'Under $10,000',
    '$5,000 to $9,999': 'Under $10,000',
    '$10,000 to $14,999': '$10,000 to $19,999',
    '$15,000 to $19,999': '$10,000 to $19,999',
    '$20,000 to $24,999': '$20,000 to $29,999',
    '$25,000 to $29,999': '$20,000 to $29,999',
    '$30,000 to $34,999': '$30,000 to $39,999',
    '$35,000 to $39,999': '$30,000 to $39,999',
    '$40,000 to $44,999': '$40,000 to $49,999',
    '$45,000 to $49,999': '$40,000 to $49,999',
	'$50,000 to $59,999': '$50,000 to $59,999',
    '$60,000 to $69,999': '$60,000 to $69,999',
    '$70,000 to $79,999': '$70,000 to $79,999',
    '$80,000 to $89,999': '$80,000 to $89,999',
    '$90,000 to $99,999': '$90,000 to $99,999',
    '$100,000 to $109,999': '$100,000 to $109,999',
    '$110,000 to $119,999': '$110,000 to $119,999',
    '$120,000 to $129,999': '$120,000 to $129,999',
    '$130,000 to $139,999': '$130,000 to $139,999',
    '$140,000 to $149,999': '$140,000 to $149,999',
    '$150,000 to $159,999': '$150,000 to $159,999',
    '$160,000 to $169,999': '$160,000 to $169,999',
    '$170,000 to $179,999': '$170,000 to $179,999',
    '$180,000 to $189,999': '$180,000 to $189,999',
    '$190,000 to $199,999': '$190,000 to $199,999',
    '$200,000 or more': '$200,000 or more',
}

# income_mapping = {
#     'Under $5,000': 'Under $50,000',
#     '$5,000 to $9,999': 'Under $50,000',
#     '$10,000 to $14,999': 'Under $50,000',
#     '$15,000 to $19,999': 'Under $50,000',
#     '$20,000 to $24,999': 'Under $50,000',
#     '$25,000 to $29,999': 'Under $50,000',
#     '$30,000 to $34,999': 'Under $50,000',
#     '$35,000 to $39,999': 'Under $50,000',
#     '$40,000 to $44,999': 'Under $50,000',
#     '$45,000 to $49,999': 'Under $50,000',
# 	'$50,000 to $59,999': '$50,000 to $99,999',
#     '$60,000 to $69,999': '$50,000 to $99,999',
#     '$70,000 to $79,999': '$50,000 to $99,999',
#     '$80,000 to $89,999': '$50,000 to $99,999',
#     '$90,000 to $99,999': '$50,000 to $99,999',
#     '$100,000 to $109,999': '$100,000 to $149,999',
#     '$110,000 to $119,999': '$100,000 to $149,999',
#     '$120,000 to $129,999': '$100,000 to $149,999',
#     '$130,000 to $139,999': '$100,000 to $149,999',
#     '$140,000 to $149,999': '$100,000 to $149,999',
#     '$150,000 to $159,999': '$150,000 to $199,999',
#     '$160,000 to $169,999': '$150,000 to $199,999',
#     '$170,000 to $179,999': '$150,000 to $199,999',
#     '$180,000 to $189,999': '$150,000 to $199,999',
#     '$190,000 to $199,999': '$150,000 to $199,999',
#     '$200,000 or more': '$200,000 or more',
# }
income_mapping_ordered = {}

# Make all new categories to their index/order.
income_values = list(income_mapping.values())
i = 0
for index, key in enumerate(income_values):
	if index == len(income_values)-1:
		income_mapping_ordered[key] = i
	elif income_values[index+1] != key:
		income_mapping_ordered[key] = i
		i += 1

income_categories = list(income_mapping_ordered.keys())

# Reset 'DEMO_household_income' column categories.
data_cross_2021['DEMO_household_income'] = data_cross_2021['DEMO_household_income'].map(income_mapping)

# Ensure 'DEMO_household_income' column is mapped to numeric values using income_mapping
data_cross_2021['DEMO_household_income_index'] = data_cross_2021['DEMO_household_income'].map(income_mapping_ordered)

# Set 'DEMO_household_income' column to be categorical with order.
data_cross_2021['DEMO_household_income'] = pd.Categorical(
    data_cross_2021['DEMO_household_income'], 
    categories=income_categories, 
    ordered=True
)

data_cross_2021

,WELLNESS_life_satisfaction,DEMO_household_income,DEMO_household_income_index
4,8.0,"$80,000 to $89,999",8
8,1.0,"$20,000 to $29,999",2
10,7.0,"$90,000 to $99,999",9
30,1.0,"Under $10,000",0
31,5.0,"$10,000 to $19,999",1
...,...,...,...
11393,4.0,"Under $10,000",0
11396,3.0,"$10,000 to $19,999",1
11398,7.0,"$30,000 to $39,999",3
11410,8.0,"$70,000 to $79,999",7


In [7]:
df_count = data_cross_2021.groupby(['DEMO_household_income', 'WELLNESS_life_satisfaction']).size().reset_index(name='Count')

# Unique sort to preserve the order of life satisfaction while also sorting the household income
for value in income_categories:
	df_count = pd.concat([df_count, df_count.loc[df_count['DEMO_household_income'] == value]], ignore_index=True)
df_count = df_count.iloc[len(df_count) // 2 :]

# Calculate the percentage directly in the DataFrame and scale it to 0-1
df_count['Percentage'] = df_count.groupby('DEMO_household_income')['Count'].transform(lambda x: x / x.sum())

# Create a stacked bar graph
fig = px.bar(df_count, x='DEMO_household_income', y='Percentage', color='WELLNESS_life_satisfaction',
             title='Percentage Bar Graph of Life Satisfaction across Household Income Brackets',
             labels={'Percentage': 'Percentage (%)',
                     'DEMO_household_income': 'Household Income Bracket',
                     'WELLNESS_life_satisfaction': 'Life Satisfaction'
                     },
             color_continuous_scale=px.colors.sequential.Plasma,
			)

# Update layout
fig.update_layout(
    barmode='stack',
    width=1000,
    height=500,
    yaxis=dict(
        tickvals=[i / 100 for i in range(0, 101, 10)],
        ticktext=[f"{i}%" for i in range(0, 101, 10)],
        dtick=0.1,  # Add grid lines every 10%
        gridcolor='rgba(0, 0, 0, 0.3)'
    ),
	margin=dict(t=80, b=80, l=80, r=80),  # Reduce the margins (top, bottom, left, right)
)

# Show the figure
fig.show()
# fig.write_image("course_project_figure1.png", format="png", engine="kaleido")

/var/folders/b0/20_s9xf13p33yqxth9cv22380000gn/T/ipykernel_1634/2453020656.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_count = data_cross_2021.groupby(['DEMO_household_income', 'WELLNESS_life_satisfaction']).size().reset_index(name='Count')
/var/folders/b0/20_s9xf13p33yqxth9cv22380000gn/T/ipykernel_1634/2453020656.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_count['Percentage'] = df_count.groupby('DEMO_household_income')['Count'].transform(lambda x: x / x.sum())


In [8]:
# Create a contingency table
contingency_table = pd.crosstab(data_cross_2021['DEMO_household_income'], 
                                 data_cross_2021['WELLNESS_life_satisfaction'])

# Run the Chi-Squared Test
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

# Print the results
print(f"Chi-Squared Statistic: {chi2}, P-value = {p}")

Chi-Squared Statistic: 239.87020096699212, P-value = 0.001894764733539881


In [9]:
simple_model = smf.ols('WELLNESS_life_satisfaction ~ C(DEMO_household_income)', data=data_cross_2021).fit()
print("Simple Model Summary:\n", simple_model.summary())

summary_str = simple_model.summary().as_text()

import matplotlib.pyplot as plt

# Create a matplotlib figure
fig, ax = plt.subplots(figsize=(10, 6))
ax.axis("off")  # Turn off the axes
ax.text(0, 1, summary_str, fontsize=10, family="monospace", verticalalignment="top")  # Add text

# Save the figure as an image
plt.savefig("simple_model_summary.png", bbox_inches="tight", dpi=300)
plt.close()

Simple Model Summary:
                                 OLS Regression Results                                
Dep. Variable:     WELLNESS_life_satisfaction   R-squared:                       0.030
Model:                                    OLS   Adj. R-squared:                  0.021
Method:                         Least Squares   F-statistic:                     3.457
Date:                        Sun, 08 Dec 2024   Prob (F-statistic):           3.35e-07
Time:                                16:00:29   Log-Likelihood:                -5158.3
No. Observations:                        2260   AIC:                         1.036e+04
Df Residuals:                            2239   BIC:                         1.048e+04
Df Model:                                  20                                         
Covariance Type:                    nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025   

In [10]:
p_values = simple_model.pvalues.drop('Intercept')

plot_data = pd.DataFrame({'Household Income Bracket': income_categories[1:], 'Correlation P-value': p_values.values})

fig = px.scatter(
    plot_data,
    x='Household Income Bracket',
    y='Correlation P-value',
    title='Correlation P-values for Household Income Brackets',
)

fig.add_scatter(
    x=plot_data['Household Income Bracket'],
    y=plot_data['Correlation P-value'],
    mode='lines',
    line=dict(color='skyblue', width=2),
    showlegend=False,
)

fig.add_hline(
    y=0.05,
    line_dash="dash",
    line_color="red",
    annotation_text="Significance Threshold (0.05)",
    annotation_position="bottom right"
)

# Modify the x-axis scale to show more numbers
fig.update_layout(
	width=800,
	height=500,
	margin=dict(t=40, b=40, l=40, r=40)  # Reduce the margins (top, bottom, left, right)
)

# Render the plot
fig.show()


In [11]:
# Extract coefficients and confidence intervals
coefficients = simple_model.params.drop('Intercept')
conf_intervals = simple_model.conf_int().drop('Intercept')

# Create the plot data with lower and upper bounds for confidence intervals
plot_data = pd.DataFrame({
    'Household Income Bracket': income_categories[1:],  # Exclude intercept
    'Coefficient': coefficients.values,
    'CI Lower': conf_intervals[0].values,  # Lower bound
    'CI Upper': conf_intervals[1].values   # Upper bound
})

# Calculate errors for the plot
plot_data['Error Upper'] = plot_data['CI Upper'] - plot_data['Coefficient']
plot_data['Error Lower'] = plot_data['Coefficient'] - plot_data['CI Lower']

# Create scatter plot with confidence intervals and flipped axes
import plotly.express as px

fig = px.scatter(
    plot_data,
    x='Household Income Bracket',
    y='Coefficient',
    title='Correlation Coefficient (with CI) for Household Income Brackets',
    error_y=plot_data['Error Upper'],
    error_y_minus=plot_data['Error Lower'],
    labels={'Coefficient': 'Correlation Coefficient'},
    width=800,
    height=500,
)

# Modify the y-axis scale to show more numbers
fig.update_layout(
    yaxis=dict(
        tickmode='linear',  # Set to linear tick mode
        tick0=-2,  # Starting tick value
        dtick=0.5,  # The distance between ticks
        automargin=True  # Automatically adjust margins on y-axis
    ),
    xaxis=dict(
        automargin=True  # Automatically adjust margins on x-axis
    ),
    margin=dict(t=80, b=80, l=80, r=80),  # Reduce the margins (top, bottom, left, right)
)

fig.add_hline(
    y=0,
    line_dash="dash",
    line_color="red",
    annotation_text="No Correlation",
    annotation_position="bottom left"
)

# Render the plot
fig.show()


In [15]:
result = pd.concat([simple_model.fittedvalues, simple_model.resid, data_cross_2021['WELLNESS_life_satisfaction']], axis=1)

# Rename columns for clarity
result.columns = ['Fitted Values', 'Residuals', 'WELLNESS_life_satisfaction']

# Display the result
print(result)

       Fitted Values  Residuals  WELLNESS_life_satisfaction
4           6.180180   1.819820                         8.0
8           5.805668  -4.805668                         1.0
10          6.404959   0.595041                         7.0
30          5.406897  -4.406897                         1.0
31          5.855422  -0.855422                         5.0
...              ...        ...                         ...
11393       5.406897  -1.406897                         4.0
11396       5.855422  -2.855422                         3.0
11398       6.219124   0.780876                         7.0
11410       6.233766   1.766234                         8.0
11430       5.805668   1.194332                         7.0

[2260 rows x 3 columns]
